# CP_ALS Profiling

#### ***Only testing cp_als.py here***

In [ ]:
import cProfile
import pstats
import os
from pyttb import ktensor, cp_als, import_data

In [ ]:
def profile(test_files, ranks):
    """
    Profiles the performance of the cp_als algorithm with a set of tensors from test_files and ranks.

    Parameters
    ----------
    test_files:
        A list of strings representing the file paths to the test tensors.
    ranks:
        A list of integers representing the tensor testing ranks.
    """

    for test_file in test_files:
        try:
            print("*" * 50)
            tensor = import_data(test_file)
            # cp_als() only handles tensors and sptensors, so convert ktensor to tensor
            if isinstance(tensor, ktensor):
                tensor = tensor.full()
            for rank in ranks:
                # initialize and enable a profiler
                profiler = cProfile.Profile()
                profiler.enable()

                try:
                    M, Minit, output = cp_als(tensor, rank)
                # ensure the profiler is always disabled before the next profiler starts
                finally:
                    profiler.disable()

                stats = pstats.Stats(profiler).sort_stats("cumulative")
                print(f"Test file: {test_file}, Rank: {rank}")
                stats.print_stats(10)
        except Exception as e:
            print(
                f"Error when testing {os.path.basename(test_file)} with Rank = {rank}: {type(e).__name__}: {e}"
            )

In [ ]:
# "../tests/data/invalid_dims.tns",
# "../tests/data/invalid_type.tns",
ranks = [2, 3, 4]
test_files = [
    "../tests/data/ktensor.tns",
    "../tests/data/matrix.tns",
    "../tests/data/sptensor_0_based.tns",
    "../tests/data/sptensor.tns",
    "../tests/data/tensor.tns",
]

In [ ]:
profile(test_files, ranks)